In [ ]:
# https://console.cloud.google.com/
# cria um projeto e ativa o Google sheets API

# Siga o quickstart abaixo para ativar a api e para criar as credenciais OAuth 2.0
# https://developers.google.com/sheets/api/quickstart/python
# Por fim baixe o arquivo com as credenciais com o nome de credentials.json 
# Deixe o arquivo credentials.json na mesma pasta deste script

In [ ]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Se der erro de escopo inválido, 
# atualize o escopo, remova o arquivo token.json e rode novamente o script

# SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly'] # ESCOPO SOMENTE LEITURA
SCOPES = ['https://www.googleapis.com/auth/spreadsheets'] # ESCOPO DE PERMISSÃO TOTAL

SAMPLE_SPREADSHEET_ID = 'ID_DA_PLANILHA'
SAMPLE_RANGE_NAME = 'Página1!A1:E1' # Range para teste de leitura
SAMPLE_RANGE_NAME2 = 'Página1!A2:E2' # Range para teste de gravação

def main():
    creds = None
    # O arquivo token.json armazena as informações de escopo e token
    # ele é criado automaticamente após a autenticação oauth ser concluída pela primeira vez
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # Se as credenciais não forem válidas o usuário deverá fazer login
    # É aberta uma página web para o usuário entrar com a conta google.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    
    # Tenta fazer a leitura    
    try: 
        service = build('sheets', 'v4', credentials=creds)
        
        # Chamada a api do google
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range=SAMPLE_RANGE_NAME).execute()
        values = result.get('values', [])
        
        if not values:
            print('No data found.')
            return
        
        # Imprime as linhas
        for row in values:
            print(row)
            
    except HttpError as err:
        print(err)
     
    
    # Tenta fazer a gravação    
    try:
        service = build('sheets', 'v4', credentials=creds)
        sheet = service.spreadsheets()
        
        values = [
            [1,2,3,4,5] # Valores que serão gravados no range 'A2:E2'
        ]
        
        body = {
            'values': values
        }
        
        result = service.spreadsheets().values().update(
            spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME2,
            valueInputOption="USER_ENTERED", body=body).execute()
        
        print(f"{result.get('updatedCells')} cells updated.")
        return result
    
    except HttpError as error:
        print(f"An error occurred: {error}")
        return error
    

if __name__ == '__main__':
    main()
    